In [ ]:
!pip install nltk
!pip install pandas
!pip install matplotlib
!pip install gensim
!pip install conllu

In [44]:
from nltk.corpus import stopwords
import nltk
from conllu import parse_incr, parse
from gensim.models import Word2Vec as w2v
import pandas as pd


In [60]:
path = '../UD_Persian-PerDT/fa_perdt-ud-dev.conllu'
data_file = open(path, "r", encoding="utf-8")
token_list = parse_incr(data_file)

sentences = []

for tokenlist in parse_incr(data_file):
    sentence = tokenlist.metadata['text']
    words = sentence.split()
    sentences.append(words)
    

In [61]:
sentences[1]

['این',
 'میهمانی',
 'به',
 'منظور',
 'آشنایی',
 'هم\u200cتیمی\u200cهای',
 'او',
 'با',
 'غذاهای',
 'ایرانی',
 'ترتیب',
 'داده',
 'شد.']

In [68]:
w = w2v(sentences=sentences, min_count=3, vector_size=50, window=3, sg=1, workers=4)

emb_df = (
    pd.DataFrame(
        [w.wv.get_vector(str(n)) for n in w.wv.key_to_index],
        index = w.wv.key_to_index
    )
)
print(emb_df.shape)
emb_df.head()

# print(w.wv.get_vector(999))
# w.wv.key_to_index


(1281, 50)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
و,0.022305,-0.050850,-0.093870,-0.054335,-0.139641,-0.066967,0.371298,0.427182,-0.353392,-0.138348,...,0.078116,0.030355,-0.078154,0.047985,0.586027,0.208841,0.013251,-0.155510,0.132532,0.077766
به,0.012468,-0.044061,-0.116701,-0.070587,-0.105144,-0.063243,0.364277,0.396518,-0.352783,-0.110412,...,0.107542,0.018636,-0.055123,0.056492,0.614524,0.225787,0.007454,-0.197965,0.120840,0.073742
را,0.002411,-0.045982,-0.096635,-0.060798,-0.107370,-0.070968,0.349568,0.415099,-0.345144,-0.141370,...,0.129320,0.003400,-0.081621,0.049419,0.578540,0.220329,0.036861,-0.181429,0.120722,0.062137
در,0.045581,-0.079703,-0.121527,-0.073514,-0.130404,-0.038378,0.387037,0.441799,-0.354198,-0.112595,...,0.116596,0.012099,-0.061174,0.042524,0.606678,0.222261,0.036460,-0.195235,0.101068,0.070071
از,0.025714,-0.048677,-0.127166,-0.079085,-0.107241,-0.034181,0.347097,0.417481,-0.361710,-0.113392,...,0.094825,0.022012,-0.063289,0.068244,0.600520,0.223748,0.034533,-0.181357,0.106658,0.055480
